In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
applications_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")


In [2]:
# how many unique values does each column have?
applications_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [3]:
# try to determine which columns could create a good model
name = applications_df['NAME'].value_counts()
name[name>40]
classification = applications_df['CLASSIFICATION'].value_counts()
classification[classification>100]
use_case = applications_df['USE_CASE'].value_counts()
use_case[use_case>100]
organization= applications_df['ORGANIZATION'].value_counts()
organization[organization>100]
income=applications_df['INCOME_AMT'].value_counts()
income[income>1000]
#con =applications_df['SPECIAL_CONSIDERATIONS'].value_counts()
#con[con>100]
ask = applications_df['ASK_AMT'].value_counts()
ask[ask<2]
active = applications_df['STATUS'].value_counts()
active[active>10]
succ= applications_df['IS_SUCCESSFUL'].value_counts()
#EIN = applications_df['EIN'].value_counts()
types = applications_df['APPLICATION_TYPE'].value_counts()
types[types>1000]
aff = applications_df['AFFILIATION'].value_counts()
aff[aff>100]


AFFILIATION
Independent         18480
CompanySponsored    15705
Name: count, dtype: int64

In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'AFFILIATION'  EIN", "AFFILIATION", "ORGANIZATION",
applications_df =  applications_df.drop(columns=[ "SPECIAL_CONSIDERATIONS", "EIN", "NAME"])

#cutoff values in columns according to findings
class_delete = list(classification[classification<100].index)
case_delete = list(use_case[use_case<100].index)
org_delete = list(organization[organization<100].index)
income_delete = list(income[income<1000].index)
#con_delete = list(con[con<100].index)
active_delete = list(active[active<10].index)
type_delete = list(types[types<1000].index)
aff_delete = list(aff[aff<100].index)
ask_delete = list(ask[ask>2].index)
#name_delete = list(name[name<100])

In [5]:
for cla in class_delete:
    applications_df['CLASSIFICATION'] = applications_df['CLASSIFICATION'].replace(cla,"Other")
#for na in name_delete:
 #   applications_df['NAME'] = applications_df['NAME'].replace(na,"Other")
for a in ask_delete:
    applications_df['ASK_AMT'] = applications_df['ASK_AMT'].replace(a,"Other")
for cas in case_delete:
    applications_df['USE_CASE'] = applications_df['USE_CASE'].replace(cas,"Other")
#for org in case_delete:
#    applications_df['ORGANIZATION'] = applications_df['ORGANIZATION'].replace(org,"Other")
for inc in income_delete:
    applications_df['INCOME_AMT'] = applications_df['INCOME_AMT'].replace(inc,"Other")
#for cons in con_delete:
#    applications_df['SPECIAL_CONSIDERATIONS'] = applications_df['SPECIAL_CONSIDERATIONS'].replace(cons,"Other")
for inc in active_delete:
    applications_df['STATUS'] = applications_df['STATUS'].replace(inc,"Other")
for types in type_delete:
    applications_df['APPLICATION_TYPE'] = applications_df['APPLICATION_TYPE'].replace(types,"Other")
#for aff in aff_delete:
#    applications_df['AFFILIATION'] = applications_df['AFFILIATION'].replace(aff,"Other")

In [6]:
#check how many unique values are left
applications_df.nunique()


APPLICATION_TYPE       6
AFFILIATION            6
CLASSIFICATION        12
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             4
ASK_AMT             8743
IS_SUCCESSFUL          2
dtype: int64

In [7]:
# convert categorical data to numeric
applications_df = pd.get_dummies(applications_df)
applications_df.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,...,ASK_AMT_1736232349,ASK_AMT_1893400128,ASK_AMT_2264109450,ASK_AMT_2310256039,ASK_AMT_3391919220,ASK_AMT_4653011914,ASK_AMT_5591584994,ASK_AMT_8556638692,ASK_AMT_8597806340,ASK_AMT_Other
0,1,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
1,1,False,False,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,0,False,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
3,1,False,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1,False,False,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [8]:
# Split our preprocessed data into our features and target arrays
X = applications_df.drop(columns=["IS_SUCCESSFUL"]).values
y = applications_df["IS_SUCCESSFUL"].values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [19]:
#see what kera thinks we should do 1/4
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(
        units=hp.Int('first_units',min_value=32,max_value=512,step=32),
        activation=activation,
        input_dim=8782))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 4)):
        nn_model.add(tf.keras.layers.Dense(
            units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])




    return nn_model

In [10]:
#2/4
# Import the kerastuner library
!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


In [20]:
#3/4

import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=8,
    hyperband_iterations=2)




Reloading Tuner from ./untitled_project/tuner0.json


In [21]:
#3/3
# Run the kerastuner search for best hyperparameters
tuner.search(X_train,y_train,epochs=8,validation_data=(X_test,y_test))



Trial 18 Complete [00h 05m 24s]
val_accuracy: 0.7279300093650818

Best val_accuracy So Far: 0.7332944869995117
Total elapsed time: 00h 55m 48s


In [23]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values



{'activation': 'relu',
 'first_units': 192,
 'num_layers': 2,
 'units_0': 7,
 'units_1': 1,
 'units_2': 5,
 'units_3': 1,
 'tuner/epochs': 8,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [24]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5739 - accuracy: 0.7333 - 1s/epoch - 5ms/step
Loss: 0.5738968253135681, Accuracy: 0.7332944869995117


In [25]:
# Compile the best model
best_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tune the best model
best_model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
804/804 [==============================] - 22s 26ms/step - loss: 0.4465 - accuracy: 0.7972 - val_loss: 0.6341 - val_accuracy: 0.6878
Epoch 2/50
804/804 [==============================] - 23s 29ms/step - loss: 0.4438 - accuracy: 0.7984 - val_loss: 0.6307 - val_accuracy: 0.6991
Epoch 3/50
804/804 [==============================] - 20s 25ms/step - loss: 0.4431 - accuracy: 0.7979 - val_loss: 0.5829 - val_accuracy: 0.7145
Epoch 4/50
804/804 [==============================] - 21s 27ms/step - loss: 0.4408 - accuracy: 0.7980 - val_loss: 0.5982 - val_accuracy: 0.7130
Epoch 5/50
804/804 [==============================] - 20s 25ms/step - loss: 0.4397 - accuracy: 0.7990 - val_loss: 0.6021 - val_accuracy: 0.7123
Epoch 6/50
804/804 [==============================] - 21s 26ms/step - loss: 0.4396 - accuracy: 0.7980 - val_loss: 0.7032 - val_accuracy: 0.6760
Epoch 7/50
804/804 [==============================] - 21s 26ms/step - loss: 0.4390 - accuracy: 0.7984 - val_loss: 0.7075 - val_accuracy:

In [26]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

NameError: name 'nn' is not defined